# Лекция 12: Map-Reduce

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2020 г. 

При подготовке лекции использованы материалы:
* ...

V 0.2 18.11.2020

## Разделы: <a class="anchor" id="разделы"></a>
* [Серии (Series) - одномерные массивы в Pandas](#серии)
* [Датафрэйм (DataFrame) - двумерные массивы в Pandas](#датафрэйм)
    * [Введение](#датафрэйм-введение)
    * [Индексация](#датафрэйм-индексация)    
* [Обработка данных в библиотеке Pandas](#обработка-данных)
    * [Универсальные функции и выравнивание](#обработка-данных-универсальные)
    * [Работа с пустыми значениями](#обработка-данных-пустрые-значения)
    * [Агрегирование и группировка](#обработка-данных-агрегирование)    
* [Обработка нескольких наборов данных](#обработка-нескольких)
    * [Объединение наборов данных](#обработка-нескольких-объединение)
    * [GroupBy: разбиение, применение, объединение](#обработка-нескольких-групбай)
 
-

* [к оглавлению](#разделы)

In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v2.css")
HTML(html.read().decode('utf-8'))

## Map / Filter / Reduce

<center>         
    <img src="./img/MFR_emj.png" alt="Иллюстрация концепции Map / Filter / Reduce" style="width: 500px;"/>
    <b>Иллюстрация концепции Map / Filter / Reduce</b>
</center>

### Map

<center>         
    <img src="./img/map_.jpg" alt="Работа функции map()" style="width: 700px;"/>
    <b>Работа функции map()</b>
</center>

Встроенная функция `map() `позволяет применить функцию к каждому элементу последовательности
* Функция имеет следующий формат: `mар(<Функция>, <Последовательность1>[, ... , <ПоследовательностьN>])`

* Функция возвращает объект, nоддерживающий итерацию, а не сnисок.

#### Map: пример 1

In [72]:
squared = lambda x: x**2

In [73]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
m1 = map(squared, range(10)) # второй аргумент - итерируемый объект!
m1 # map реализует принцип ленивых вычислений

In [8]:
list(m1)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

#### Map: пример 2

In [1]:
ls0 = list(zip(range(10), range(0, 100, 10)))
ls0

[(0, 0),
 (1, 10),
 (2, 20),
 (3, 30),
 (4, 40),
 (5, 50),
 (6, 60),
 (7, 70),
 (8, 80),
 (9, 90)]

In [74]:
import operator as op

In [75]:
op.add(2, 3)

5

In [4]:
op.add((2, 3))

TypeError: add expected 2 arguments, got 1

In [76]:
op.add(*(2, 3))

5

применение функции, принимающей несколько параметров:

In [6]:
list(map(op.add, ls0)) # ошибка, неверное количество параметров!

TypeError: add expected 2 arguments, got 1

In [77]:
# 1й способ:
def my_add(par):
    return op.add(*par)

In [78]:
list(map(my_add, ls0)) 

[0, 11, 22, 33, 44, 55, 66, 77, 88, 99]

In [79]:
# 2й способ:

from itertools import starmap
# starmap вычисляет значение функции для аргументов сгруппированных в итерируемом объекте (втром параметре)

In [80]:
list(starmap(op.add, ls0))

[0, 11, 22, 33, 44, 55, 66, 77, 88, 99]

#### Map: пример 3

<center>         
    <img src="./img/map2_.jpg" alt="Работа функции map() с несколькими итерируемыми объектами" style="width: 700px;"/>
    <b>Работа функции map() с несколькими итерируемыми объектами</b>
</center>

Функции map() можно nередать несколько nоследовательностей. В этом случае в функцию 
обратного вызова будут nередаваться сразу несколько элементов, расnоложенных в nоследовательностях на одинаковом смещении. 

In [11]:
ls1 = list(range(10))
ls2 = list(range(0, 100, 10))
ls3 = list(range(0, 1000, 100))

In [12]:
ls1, ls2, ls3

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90],
 [0, 100, 200, 300, 400, 500, 600, 700, 800, 900])

In [13]:
list(map(op.add, ls1, ls2))

[0, 11, 22, 33, 44, 55, 66, 77, 88, 99]

In [14]:
list(map(lambda x, y, z: x+10*y+100*z, ls1, ls2, ls3))

[0, 10101, 20202, 30303, 40404, 50505, 60606, 70707, 80808, 90909]

### Filter

<center>         
    <img src="./img/filter_.jpg" alt="Работа функции filter()" style="width: 700px;"/>
    <b>Работа функции filter()</b>
</center>

Функция `filter()` nозволяет выnолнить nроверку элементов nоследовательности. 
* Формат функции: `filtеr(<Функция>, <Последовательность>)`
* Если в nервом nараметре вместо названия функции указать значение `None`, то каждый элемент nоследонательности будет nроверен на соответствие булевскому значению True.
* Если элемент в логическом контексте возвращает значение False, то он не будет добавлен в возвращаемый результат. 
* Функция возвращает объект, nоддерживающий итерацию, а не сnисок.

In [15]:
import random
random.seed(42)

In [81]:
lr1 = [random.randint(-100, 100) for i in range(20)]
lr1

[-29,
 -61,
 -45,
 95,
 -14,
 -74,
 -77,
 -3,
 -76,
 -9,
 -12,
 54,
 -33,
 -89,
 86,
 17,
 37,
 -69,
 -4,
 -80]

In [82]:
list(filter(lambda x: x%3 == 0, lr1))

[-45, -3, -9, -12, 54, -33, -69]

In [83]:
# аналог генератор списков:
[i for i in lr1 if i%3==0]

[-45, -3, -9, -12, 54, -33, -69]

In [84]:
lr2 = [random.randint(-1, 1) for i in range(20)]
lr2

[1, 0, 1, 1, 0, 1, -1, 1, -1, -1, 1, -1, 0, -1, -1, -1, 0, 0, 0, 1]

In [85]:
# первый параметр None имеет особую семантику:
list(filter(None, lr2))

[1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1]

In [86]:
filter(lambda x: x%3 == 0, lr1)

In [87]:
map(op.abs, filter(lambda x: x%3 == 0, lr1))

In [88]:
# последовательное примененеие преобразований:
list(map(op.abs, filter(lambda x: x%3 == 0, lr1)))

[45, 3, 9, 12, 54, 33, 69]

In [33]:
%%timeit
[i for i in lr1 if i%3==0]

2 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%%timeit
list(filter(lambda x: x%3 == 0, lr1))

4.15 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Reduce

In [26]:
import functools

In [27]:
from functools import reduce

`functools.reduce(funct, iterable[, initializer])`

Вычисляет функцию от двух элементов последовательно для элементов последовательности слева направо таким образом, что результатом вычисления становится единственное значение, которое становится первым аргументом для следующей итерации применения `funct`. 

In [28]:
# Пример: 
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5]) # вычисляется как ((((1+2)+3)+4)+5)

15

Левый аргумент функции `funct` (аргумента `reduce`) - это аккумулированное значение, правый аргумент - очередное значение из списка.

Если передан необязательный аргумент `initializer`, то он используется в качестве левого аргумента при первом применении 
функции (исходного аккумулированного значения).

Если `initializer` не перередан, а последовательность имеет только одно значение, то возвращается это значенние.

<center>         
    <img src="./img/reduce_.jpg" alt="Работа функции reduce()" style="width: 700px;"/>
    <b>Работа функции reduce()</b>
</center>

In [89]:
ls4 = list(range(10, 20))
ls4

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [90]:
reduce(op.add, ls4)

145

In [91]:
def add_verbose(x, y):
    print("add(x=%s, y=%s) -> %s" % (x, y, x+y))
    return x + y

In [93]:
reduce(add_verbose, ls4)

add(x=10, y=11) -> 21
add(x=21, y=12) -> 33
add(x=33, y=13) -> 46
add(x=46, y=14) -> 60
add(x=60, y=15) -> 75
add(x=75, y=16) -> 91
add(x=91, y=17) -> 108
add(x=108, y=18) -> 126
add(x=126, y=19) -> 145


145

In [34]:
reduce(add_verbose, ls4, 1000)

add(x=1000, y=10) -> 1010
add(x=1010, y=11) -> 1021
add(x=1021, y=12) -> 1033
add(x=1033, y=13) -> 1046
add(x=1046, y=14) -> 1060
add(x=1060, y=15) -> 1075
add(x=1075, y=16) -> 1091
add(x=1091, y=17) -> 1108
add(x=1108, y=18) -> 1126
add(x=1126, y=19) -> 1145


1145

In [94]:
st = "This is a test.".split()
st

['This', 'is', 'a', 'test.']

In [95]:
def f2(n, s):
    print(f'n: {n}, s: {s}, len(s): {len(s)}')
    return n + len(s)

In [96]:
# Ошибка:
reduce(f2, ['This', 'is', 'a', 'test.']) 

n: This, s: is, len(s): 2


TypeError: can only concatenate str (not "int") to str

In [97]:
reduce(f2, ['This', 'is', 'a', 'test.'], 0) 

n: 0, s: This, len(s): 4
n: 4, s: is, len(s): 2
n: 6, s: a, len(s): 1
n: 7, s: test., len(s): 5


12

In [39]:
reduce(lambda n, s: n + len(s), "This is a test.".split(), 0) 

12

In [40]:
reduce(lambda n, s: n + s, "This is a test.".split(), "") 

'Thisisatest.'

## Dask Bag

### Принципы Dask Bag

__Структура данных Bag__

<em class="df"></em> __Мультимножество (bag, multiset)__ в математике - обобщение понятия множества, допускающее включение одного и того же элемента по нескольку раз. Число элементов в мультимножестве, с учётом повторяющихся элементов, называется его размером или мощностью.

* `list`: упорядоченная коллекция, допускающая повторы элементв.
    * Пример: `[1, 2, 3, 2]`
* `set`: неупорядоченная коллекция, не допускающая повторы элементов.
    * Пример: `{1, 2, 3}`
* `bag`: неупорядоченная коллекция, допускающая повторы элементов. 
    * Пример: `1, 2, 2, 3`

Таким образом, bag можно рассматривать как __список, не гарантирующий порядка элементов__.

__Dask.Bag__

`Dask.Bag` реализует такие операции, как `map`, `filter`, `fold` (аналог `reduce`) и `groupby` над коллекциями объектов Python. 

Реализация `Dask.Bag` основана на координации множества списков или итераторов, каждый из которых представляет собой сегмент большой коллекции. Данная реализация обеспечивает: 
* параллельное выполнение операций 
* потребность в небольшом объеме памяти за счет использования итераторов Python и __ленивых вычислений__. Это обеспечивает возможность обработки данных больших чем объем оперативной памяти, даже при использовании всего одного сегмента.


<em class="df"></em> __Ленивые вычисления__ (lazy evaluation, или отложенные вычисления) — стратегия вычислений, согласно которой вычисления откладываются до тех пор, пока не понадобится их результат.

Аналоги:
* `Dask.Bag` можно считать параллельной реализацией пакета `PyToolz` 
* Или ориентированной на Python версией `PySpark RDD` (интерфейса для работы в Python с ключевой структурой данных Spark - RDD).

__Типичное использование Dask.Bag__

`Dask.Bag` хорошо подходит для распараллеливания __простой обработки неструктурированных или полу-структурированных данных__, таких как:
* текстовые данные
* файлы логирования
* записи в формате JSON
* специальных oбектов Python и т.д. 

Если выполнение задачи возможно при помощи `Dask.DataFrame` или `Dask.Array`, то стоит выбрать этии варианты, так как основной объем вычислений будет выполняться за счет быстрых библиотек написанных на компилируемых языках, тогда как `Dask.Bag` использует только код на Python. При этом приемуществом `Dask.Bag` является возможность использовать любые пользовательские функции написанные на Python и существенно меньшие требования к наличию строгой структуры у обрабатываемых данных.

__Специфика реализации__

По умолчанию, `Dask.Bag` использует для исполнения __планировщик__ `dask.multiprocessing`.
* <em class="pl"></em> Это позволяет __обойти проблему GIL__ и полноценно использовать несколько процессорных ядер для объектов реализованных на чситом Python.
* <em class="mn"></em> Минусом этого подходя является наличие больших накладных расходов при обмене данных между исполнителями, что важно для производительности вычислений, требующих интенсивного обмена данными. Это редко бывает проблемой, так как типичный поток задач для `Dask.Bag` подразумевает:
    * или черезвычайно параллельные вычисления 
    * или обмен небольшим объемом данных в процессе __свертки__ (англ. folding, также известна как reduce, accumulate).

<em class="df"></em> __Чрезвычайная параллельность__ (embarrassingly parallel) - тип задач в системах параллельных вычислений, для которых не требуется прилагать больших усилий при разделении на несколько отдельных параллельных задач (распараллеливании).
* Чаще всего __не существует зависимости (или связи) между  параллельными задачами__, то есть их результаты не влияют друг на друга.
* Чрезвычайно параллельные задачи __практически не требуют согласования__ между результатами выполнения отдельных этапов, что отличает их от задач распределённых вычислений, которые требуют связи промежуточных результатов.
* Такие задачи __легки для исполнения массово паралельных системах__ (кластерах с очень большим количеством вычислительных узлов).

В модели вычислений MapReduce шаг "перетасовывания" (shuffle), отвечающий за группировку данных, требует интенсивного обмена данными между исполнителями.
* Например, шаг группировки выполняется при выполнении операции `Dask.Bag` `groupby`.
* Операция группировки очень ресурсоемкая и эффективнее выполняется при помощи `Dask.DataFrame` или `Dask.Array`. Поэтму предпочтительнее использовать `Dask.Bag` для подготовки и структурирования данных, а для выполнения более сложных операций преобразовывать их в `Dask.DataFrame`.

__Ограничения Dask.Bag__

Dask.Bag позволяет выполнять любую функцию на Python, эта универсальность имеет свою цену. Dask.Bag имеет следующие ограничения:
* по умолчанию обработка Dask.Bag выполняется планировщиком, на базе multiprocessing, что создает ряд ограничений
* Bag яляется __неизменяемой структурой данных__, таким образом нет возможности изменить единичный элемент Dask.Bag не выполнив операцию преобразования для всей структуры данных
* операции над Bag медленнее, чем операции над Array/DataFrame по тем же причинам, по которым операции на Python медленнее чем аналогичные операции в NumPy и Pandas 
* операция __Bag.groupby выполняется медленно__, по возможности нужно использовать вместо нее Bag.foldby или преобразовывать данные в структуру DataFrame.

### Создание Dask Bag

`Bag` можно создать из последовательности (итерируемого объекта) Python, из файла (файлов) и т.п. Данные в `Bag` разбиваются на сегменты (блоки), каждый из которых содержит множество элементов исходного набора данных.

In [98]:
import dask.bag as db

In [99]:
# набор данных (list - итерируемый объект Python) из целых чисел
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2) # разбиваем последовательность на 2 сегмента
b.take(3) # выводит 3 элемента из Bag

(1, 2, 3)

Через параметр `npartitions` можно управлять разбиением данных на сегменты (partitions). По умолчанию Dask пытается разбить данные на количество сегментов, близкое к 100.

Функция `read_text` может принимать на вход файл или последовательность файлов:

```python
>>> b = db.read_text('myfile.txt')
>>> b = db.read_text(['myfile.1.txt', 'myfile.2.txt', ...])
>>> b = db.read_text('myfile.*.txt')
```

* Функция умеет использовать стандартные форматы архивов, такие как gzip, bz2, xz (могут быть подключены и другие форматы).

* Элементами в созданном Bag будут строки переданных текстовых файлов. По умолчанию каждый сегмент Bag будет относится к одному файлу (при помощи параметров files_per_partition и blocksize можно управлять разбиением на сегменты). 

* __Не загружайте данные предназначенные для Bag предварительно__, используйте для этого только функции Bag. Это позволит избежать нераспараллеленного участка работ и загрузки всего массива данных в память. Если для используемого формата данных нет стандартного загрузчика - исползуйте следующий подход:

```python
>>> # load_from_filename - пользовательская функция загрузки данных
>>> b = db.from_sequence(['1.dat', '2.dat', ...]).map(load_from_filename)
```

In [58]:
# Генерация данных (нужно создать папку ./data)
# скрипты для генерации (в т.ч. prep.py лежат в текущей папке)
%run prep.py -d accounts

In [100]:
# данные загружаются из набора заархивированных файлов формата JSON
# элементами Bag будут строки файлов, сегментация будет проведена по файлам
data_path = './data'
import os
b = db.read_text(os.path.join(data_path, 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Frank", "transactions": [{"transaction-id": 1341, "amount": 289}, {"transaction-id": 3824, "amount": 240}, {"transaction-id": 5168, "amount": 235}, {"transaction-id": 7303, "amount": 250}, {"transaction-id": 7580, "amount": 283}, {"transaction-id": 9440, "amount": 169}, {"transaction-id": 9660, "amount": 127}, {"transaction-id": 10265, "amount": 247}, {"transaction-id": 10680, "amount": 171}, {"transaction-id": 10864, "amount": 142}, {"transaction-id": 11473, "amount": 183}, {"transaction-id": 12037, "amount": 203}, {"transaction-id": 12272, "amount": 206}, {"transaction-id": 12767, "amount": 156}, {"transaction-id": 13008, "amount": 195}, {"transaction-id": 13233, "amount": 162}, {"transaction-id": 13614, "amount": 197}, {"transaction-id": 14379, "amount": 240}, {"transaction-id": 15409, "amount": 313}, {"transaction-id": 16717, "amount": 162}, {"transaction-id": 16873, "amount": 221}, {"transaction-id": 19635, "amount": 162}, {"transaction-id": 21944, "amount": 1

In [101]:
b.npartitions

50

### API Dask.Bag

Объекты Bag поддерживают стандартное API, аналогичное имеющемуся в стандартной библиотеке Python и библиотеках toolz или pyspark. В частности, имеются функции, отвечающие за маппинг (map и т.п.), фильтрацию и группировку (filter, groupby и т.п.) и свертку (reduce и т.п.).

Документация по API Dask.Bag : https://docs.dask.org/en/latest/bag-api.html 

In [102]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda-..., npartitions=10>

* Операции над объектом Bag, создают новые объекты Bag, таким образом формируются задачи для отоложенных вычислений. 
* Для старта вычислений необходимо вызвать для объекта Bag функцию `compute()`.
    * Результат `compute()` для объектов Bag будет представлен в виде списка (или единичного значения при операциях свертки).
    * Получение итератора по bag приводит к выполнению `compute()`. Таким образмом `list(bag)` автоматически стартует вычисления.

In [103]:
# вызов compute() синхронный: возрват из функции происхоит только после получения всех результатов
c.compute()

[4, 16, 36, 64, 100]

In [55]:
list(b.filter(is_even).map(lambda x: x ** 2))

[4, 16, 36, 64, 100]

Для того чтобы стартовать промежуточные вычсиления, которые приведут к созданию объекта Dask (например Bag), хранящегося в памяти (например загрузки данных из файла и их предварительной обработки) вместо `compute()` нужно вызывать функцию `persist()`. С полученным объектом Bag можно выполнять последующие операции исходя из того что предварительные операции уже выполнены и результаты уже хранятся в оперативной памяти. 

In [104]:
p = c.persist()

In [50]:
m1 = p.map(lambda x: x / 3)

In [106]:
m1

dask.bag<lambda-..., npartitions=10>

In [107]:
list(m1)

[1.3333333333333333,
 5.333333333333333,
 12.0,
 21.333333333333332,
 33.333333333333336]

Если после обработки с помощью Bag получены хорошо структурированные данные, то продолжить их обработку может буть эффективнее при помощи Dask DataFrame (аналог Pandas DataFrame). Ряд операций для DataFrame реализованы существенно эффективнее, да и обычные функции, за счет вызовов откомпелированного кода Pandas, выполняются намного быстрее.

In [108]:
b = db.from_sequence([{'name': 'Alice',   'balance': 100}, 
                      {'name': 'Bob',     'balance': 200},
                      {'name': 'Charlie', 'balance': 300}], npartitions=2)
# преобразование в Dask DataFrame:
df = b.to_dataframe()
df

,name,balance
npartitions=2,,
,object,int64
,...,...
,...,...


__Функции маппинга__

Функции маппинга для DaskBag:

| Функция | Краткое описание |
|------|------|
|Bag.map(func, \*args, \*\*kwargs) | Apply a function elementwise across one or more bags|
|Bag.map_partitions(func, \*args, \*\*kwargs) | Apply a function to every partition across one or more bags|
|Bag.starmap(func, \*\*kwargs) | Apply a function using argument tuples from the given bag|
|Bag.pluck(key[, default]) | Select item from all tuples/dicts in collection|

In [109]:
import operator as op

Примеры использования функции `map`

In [110]:
b1 = db.from_sequence(range(6), npartitions=2)
b1.map(lambda x: x + 1).compute()

[1, 2, 3, 4, 5, 6]

In [111]:
b2 = db.from_sequence(range(6, 12), npartitions=2)
# после func в map могут передваться другие объекты Bag для выполнения мэппинга с несколькими мультимножествами
# все объекты Bag, участвующие в мэппинге должны быть одинаково сегментированны
b1.map(op.add, b2).compute()

[6, 8, 10, 12, 14, 16]

In [114]:
b3 = db.from_sequence([0, 3, 15]*2, npartitions=2)
# использование 3х Bag'ов в map:
b1.map(lambda *x: sum(x)/len(x), b2, b3).compute()

[2.0,
 3.6666666666666665,
 8.333333333333334,
 4.0,
 5.666666666666667,
 10.333333333333334]

In [115]:
# Дополнительные аргументы (позиционные и именованные) для func могут передваться через *args, **kwargs
# Аргументы будут транслироваться для всех вызовов func. 
# Дополнительные аргументы должны следовать после объектов типа Bag.
b1.map(op.pow, 3).compute()

[0, 1, 8, 27, 64, 125]

Примеры использования функции `starmap`

In [69]:
# starmap применяет func к картежам аргументов из преданного на вход объекта bag
b = db.from_sequence([(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)], npartitions=2)
b.starmap(op.add).compute()

[3, 7, 11, 15, 19]

Примеры использования функции `pluck`

In [118]:
# pluck выбирает значение по ключу (индексу) из коллекции (словаря, списка, картежа)

b = db.from_sequence([{'name': 'Alice', 'credits': [1, 2, 3]},
                      {'name': 'Bob',   'credits': [10, 20]},
                      {'name': 'Rob'}])

In [119]:
b.map(lambda x: x['name']).compute()

['Alice', 'Bob', 'Rob']

In [120]:
b.pluck('name').compute()

['Alice', 'Bob', 'Rob']

In [123]:
# использование значения по умолчанию:
b.pluck('credits', default=[-1]).compute()

[[1, 2, 3], [10, 20], [-1]]

In [124]:
# работа с bag из списков:
b = db.from_sequence([[1, 2, 3], [10, 20], [11, 7]])

In [125]:
b.map(lambda x: x[1]).compute()

[2, 20, 7]

In [126]:
b.pluck(1).compute()

[2, 20, 7]

__Преобразование строк__

Преобразование строк для Dask Bag можно выполнять с использованием функций из пространства имен `str`.

Обработку строк, находящихся в объектах Bag, можно осуществлять используя пространство имен `str`, которое напрямую привязано к объектам Bag. Таким образом мэппинг функций из str можно производить напрямую, не исопльзуя функцию `map`.

In [128]:
b = db.from_sequence(['Alice Smith', 'Bob Jones', 'Charlie Smith'])

In [79]:
b.map(lambda x: x.lower()).compute()

['alice smith', 'bob jones', 'charlie smith']

In [129]:
print(b.str.lower().compute())

print(b.str.match('*Smith').compute())

print(b.str.split(' ').compute())

['alice smith', 'bob jones', 'charlie smith']
['Alice Smith', 'Charlie Smith']
[['Alice', 'Smith'], ['Bob', 'Jones'], ['Charlie', 'Smith']]


__Функции фильтрации__

| Функция | Краткое описание|
|------|------|
|Bag.filter(predicate) | Filter elements in collection by a predicate function|
|Bag.random_sample(prob[, random_state]) | Return elements from bag with probability of prob|
|Bag.remove(predicate) | Remove elements in collection that match predicate|

Примеры использования функции `filter`

In [61]:
b = db.from_sequence(range(5))
list(b.filter(lambda x: x % 2 == 0))  

[0, 2, 4]

Примеры использования функции `random_sample`

In [62]:
b = db.from_sequence(range(50))
list(b.random_sample(0.1))

[2, 5, 28, 40]

Примеры использования функции `remove`

In [63]:
b = db.from_sequence(range(5))
# удаляет все элементы для которых выполняется предикат
list(b.remove(lambda x: x % 2 == 0))

[1, 3]

__Функции, преобразующие Bag__

| Функция | Краткое описание|
|------|------|
|concat(bags) | Concatenate many bags together, unioning all elements|
|zip(\*bags) | Partition-wise bag zip|
|||
|Bag.join(other, on_self[, on_other]) | Joins collection with another collection|
|Bag.product(other) | Cartesian product between two bags|
|Bag.flatten() | Concatenate nested lists into one long list|
|Bag.repartition(npartitions) | Coalesce bag into fewer partitions|

Примеры использования функции `concat`

In [64]:
a = db.from_sequence([1, 2, 3])
b = db.from_sequence([4, 5, 6])
print(a.npartitions)
c = db.concat([a, b])
print(c.npartitions)
list(c)

3
6


[1, 2, 3, 4, 5, 6]

Примеры использования функции `zip`

In [65]:
# переданные мультимножества должны иметь одинаковое количество сегментов, 
# а количество элементов в сегментах должно совпадать
evens = db.from_sequence(range(0, 20, 2), partition_size=4)
odds = db.from_sequence(range(1, 20, 2), partition_size=4)
pairs = db.zip(evens, odds)
list(pairs)

[(0, 1),
 (2, 3),
 (4, 5),
 (6, 7),
 (8, 9),
 (10, 11),
 (12, 13),
 (14, 15),
 (16, 17),
 (18, 19)]

In [66]:
evens.npartitions, pairs.npartitions

(3, 3)

Примеры использования функции `join`

In [67]:
people = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Robert'])
fruit_l = ['Apple', 'Apricot', 'Banana', 'Fig', 'Tangerine']

people.npartitions

4

In [68]:
# bag.join выполняет соединение со значениями  из итерируемого контейнера 
# по условию равенства результатов возвращаемых функцией on_self
# соединение слов равной длины:
list(people.join(fruit_l, len)) 

[('Apple', 'Alice'),
 ('Fig', 'Bob'),
 ('Apricot', 'Charlie'),
 ('Banana', 'Robert')]

In [70]:
# соединение слов, начинающихся на одинаковую букву:
list(people.join(fruit_l, lambda x: x[0])) 

[('Apple', 'Alice'), ('Apricot', 'Alice'), ('Banana', 'Bob')]

In [71]:
# соединение слов, которые в people заканичваются на букву с которой начинаются слова в fruit:
list(people.join(fruit_l, lambda x: x[-1], lambda x: x[0].lower())) 

[('Banana', 'Bob'), ('Tangerine', 'Robert')]

Примеры использования функции `product`

In [96]:
fruit = db.from_sequence(fruit_l[:3])
# возвращает bag содержащий все пары (в виде картежей) элементов исходных мультимножеств 
# элементы в картежах упорядочены согласно порядку в операции исходных мультимножест 
pp = people.product(fruit)
print(people.npartitions)
list(pp)

4


[('Alice', 'Apple'),
 ('Alice', 'Apricot'),
 ('Alice', 'Banana'),
 ('Bob', 'Apple'),
 ('Bob', 'Apricot'),
 ('Bob', 'Banana'),
 ('Charlie', 'Apple'),
 ('Charlie', 'Apricot'),
 ('Charlie', 'Banana'),
 ('Robert', 'Apple'),
 ('Robert', 'Apricot'),
 ('Robert', 'Banana')]

In [95]:
pp.npartitions

12

Примеры использования функции `flatten`

In [97]:
people.map(list).compute()

[['A', 'l', 'i', 'c', 'e'],
 ['B', 'o', 'b'],
 ['C', 'h', 'a', 'r', 'l', 'i', 'e'],
 ['R', 'o', 'b', 'e', 'r', 't']]

In [100]:
people.npartitions

4

In [101]:
# конкатенирует вложнные списки в плоское (без вложенных контейнеров) мультимножество
people_flat = people.map(list).flatten()
print(people_flat.npartitions)
print(list(people_flat))

4
['A', 'l', 'i', 'c', 'e', 'B', 'o', 'b', 'C', 'h', 'a', 'r', 'l', 'i', 'e', 'R', 'o', 'b', 'e', 'r', 't']


__Функции группировки (shuffle)__

| Функция | Краткое описание|
|------|------|
|Bag.groupby(grouper[, method, npartitions, …]) | Group collection by key function|
|Bag.distinct() | Distinct elements of collection|
|Bag.frequencies([split_every, sort]) | Count number of occurrences of each distinct element|\

Примеры использования функции `groupby`

In [102]:
b = db.from_sequence(range(10))
# группирует значения из bag по результатам функции key
# очень ресурсоемкая операция, по возможности нужно заменять функцие foldby, совмещающей группировку со сверткой
list(b.groupby(lambda x: x % 2 == 0))  

[(False, [7, 9, 1, 3, 5]), (True, [8, 0, 4, 2, 6])]

Примеры использования функции `distinct`

In [103]:
b = db.from_sequence(['Alice', 'Bob', 'Alice'])
# возвращает уникальные значения из bag, не гарантирует упорядоченности..............э
list(b.distinct())

['Alice', 'Bob']

Примеры использования функции `frequencies`

In [104]:
# подсчитывает частоты для элементов bag
b = db.from_sequence(['Alice', 'Bob', 'Alice'])
list(b.frequencies()) 

[('Alice', 2), ('Bob', 1)]

__Функции свертки__

| Функция | Краткое описание |
|------|------|
|Bag.fold(binop[, combine, initial, split_every]) | Parallelizable reduction|
|Bag.foldby(key, binop[, initial, combine, …]) | Combined reduction and groupby|
|Bag.reduction(perpartition, aggregate[, …]) | Reduce collection with reduction operators|




Примеры использования функции `fold`

In [105]:
b = db.from_sequence(range(5))
# fold - параллельная версия reduce, binop - бинарный оператор, используемый для свертки
b.fold(op.add).compute()

10

In [106]:
text = db.from_sequence('Fold is like the builtin function reduce except that it works in parallel'.split())
# bag со списками букв, содержащихся в словах
text = text.map(list)
# list(text)

In [107]:
def add_to_set(acc, x):
    return acc | set(x)

# fold может использовать две функции свертки на разных этапах операции
# первый параметр: функция для свертки на уровене сегмента; второй - для свертки результататов, полученных в сегментах
# возвращает множество всех букв, встречающихся в тексте:
print(text.fold(add_to_set, set.union, initial=set()).compute())

{'s', 'b', 'f', 'p', 'w', 'e', 'n', 'k', 'd', 'o', 'l', 'u', 'x', 'F', 'r', 'i', 't', 'a', 'c', 'h'}


Примеры использования функции `foldby`

`foldby` комбинирует свертку и группировку и выполняет эту операцию намного эффективнее последовательного применения `groupby` и `reduce`.

Последовательный аналог задачи, решаемой функцией `foldby`:
```python
>>> def reduction(group):                               
...     return reduce(binop, group, init) 

>>> b.groupby(key).map(lambda (k, v): (k, reduction(v)))
```

In [108]:
b = db.from_sequence(range(10))
iseven = lambda x: x % 2 == 0
# параметр key - опеределение ключа для группировки, binop - функция для проведения свертки
# сумма четных и нечетных числе из bag:
list(b.foldby(iseven, op.add))

[(True, 20), (False, 25)]

__Функции выполняющие агрегацию (свертку с заданными функциями)__

| Функция | Краткое описание |
|------|------|
|Bag.all([split_every]) | Are all elements truthy?|
|Bag.any([split_every]) | Are any of the elements truthy?|
|Bag.count([split_every]) | Count the number of elements|
|Bag.max([split_every]) | Maximum element|
|Bag.mean() | Arithmetic mean|
|Bag.min([split_every]) | Minimum element|
|Bag.std([ddof]) | Standard deviation|
|Bag.var([ddof])| Variance|
|Bag.sum([split_every]) | Sum all elements|

Свертка возвращающая несколько значений 

| Функция | Краткое описание |
|------|------|
|Bag.take(k[, npartitions, compute, warn])|Take the first k elements|
|Bag.topk(k[, key, split_every]) | K largest elements in collection|


Примеры использования функции `sum`

In [109]:
b = db.from_sequence(range(10))
b.sum().compute()

45

Примеры использования функции `take`

In [110]:
'Aa'.lower()

'aa'

In [111]:
text = db.from_sequence('Fold is like the builtin function reduce except that it works in parallel'.split(), npartitions=1)
# возвращает первые k элементов bag
# по умолчанию значение параметра compute True, т.е. автоматически выполняются отложенные вычисления
# по умолчанию работает только с первым сегментом bag
text.str.lower().take(3)

('fold', 'is', 'like')

Примеры использования функции `topk`

In [112]:
text = db.from_sequence('Fold is like the builtin function reduce except that it works in parallel'.split())
# k наибольших значений bag
# в некоторых задачах использование функции может заменить сортировку (не реализованную для Dask Bag)
list(text.str.lower().topk(3))

['works', 'the', 'that']

In [113]:
text = db.from_sequence('Fold is like the builtin function reduce except that it works in parallel'.split())

# функция key определяет пользовательский ключ для сортировки
list(text.topk(3, key=len))

['function', 'parallel', 'builtin']

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [124]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Frank", "transactions": [{"transaction-id": 1341, "amount": 289}, {"transaction-id": 3824, "amount": 240}, {"transaction-id": 5168, "amount": 235}, {"transaction-id": 7303, "amount": 250}, {"transaction-id": 7580, "amount": 283}, {"transaction-id": 9440, "amount": 169}, {"transaction-id": 9660, "amount": 127}, {"transaction-id": 10265, "amount": 247}, {"transaction-id": 10680, "amount": 171}, {"transaction-id": 10864, "amount": 142}, {"transaction-id": 11473, "amount": 183}, {"transaction-id": 12037, "amount": 203}, {"transaction-id": 12272, "amount": 206}, {"transaction-id": 12767, "amount": 156}, {"transaction-id": 13008, "amount": 195}, {"transaction-id": 13233, "amount": 162}, {"transaction-id": 13614, "amount": 197}, {"transaction-id": 14379, "amount": 240}, {"transaction-id": 15409, "amount": 313}, {"transaction-id": 16717, "amount": 162}, {"transaction-id": 16873, "amount": 221}, {"transaction-id": 19635, "amount": 162}, {"transaction-id": 21944, "amount": 1

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [125]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Frank',
  'transactions': [{'transaction-id': 1341, 'amount': 289},
   {'transaction-id': 3824, 'amount': 240},
   {'transaction-id': 5168, 'amount': 235},
   {'transaction-id': 7303, 'amount': 250},
   {'transaction-id': 7580, 'amount': 283},
   {'transaction-id': 9440, 'amount': 169},
   {'transaction-id': 9660, 'amount': 127},
   {'transaction-id': 10265, 'amount': 247},
   {'transaction-id': 10680, 'amount': 171},
   {'transaction-id': 10864, 'amount': 142},
   {'transaction-id': 11473, 'amount': 183},
   {'transaction-id': 12037, 'amount': 203},
   {'transaction-id': 12272, 'amount': 206},
   {'transaction-id': 12767, 'amount': 156},
   {'transaction-id': 13008, 'amount': 195},
   {'transaction-id': 13233, 'amount': 162},
   {'transaction-id': 13614, 'amount': 197},
   {'transaction-id': 14379, 'amount': 240},
   {'transaction-id': 15409, 'amount': 313},
   {'transaction-id': 16717, 'amount': 162},
   {'transaction-id': 16873, 'amount': 221},
   {'transaction

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [126]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 12,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1284, 'amount': 28},
   {'transaction-id': 2262, 'amount': 24},
   {'transaction-id': 4392, 'amount': 25},
   {'transaction-id': 4633, 'amount': 25},
   {'transaction-id': 9418, 'amount': 25},
   {'transaction-id': 9795, 'amount': 21},
   {'transaction-id': 11345, 'amount': 22},
   {'transaction-id': 11406, 'amount': 24},
   {'transaction-id': 11480, 'amount': 25},
   {'transaction-id': 12164, 'amount': 22},
   {'transaction-id': 12738, 'amount': 27},
   {'transaction-id': 13076, 'amount': 21},
   {'transaction-id': 13182, 'amount': 24},
   {'transaction-id': 14485, 'amount': 23},
   {'transaction-id': 15001, 'amount': 24},
   {'transaction-id': 17278, 'amount': 23},
   {'transaction-id': 17666, 'amount': 26},
   {'transaction-id': 18117, 'amount': 22},
   {'transaction-id': 20855, 'amount': 23},
   {'transaction-id': 21198, 'amount': 24},
   {'transaction-id': 25098, 'amount': 24},
   {'transaction-id': 27115, 'amoun

In [127]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 66},
 {'name': 'Alice', 'count': 111},
 {'name': 'Alice', 'count': 53},
 {'name': 'Alice', 'count': 123},
 {'name': 'Alice', 'count': 63})

In [128]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(66, 111, 53, 123, 63)

In [129]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

120.71933333333334

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [130]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 1284, 'amount': 28},
  {'transaction-id': 2262, 'amount': 24},
  {'transaction-id': 4392, 'amount': 25},
  {'transaction-id': 4633, 'amount': 25},
  {'transaction-id': 9418, 'amount': 25},
  {'transaction-id': 9795, 'amount': 21},
  {'transaction-id': 11345, 'amount': 22},
  {'transaction-id': 11406, 'amount': 24},
  {'transaction-id': 11480, 'amount': 25},
  {'transaction-id': 12164, 'amount': 22},
  {'transaction-id': 12738, 'amount': 27},
  {'transaction-id': 13076, 'amount': 21},
  {'transaction-id': 13182, 'amount': 24},
  {'transaction-id': 14485, 'amount': 23},
  {'transaction-id': 15001, 'amount': 24},
  {'transaction-id': 17278, 'amount': 23},
  {'transaction-id': 17666, 'amount': 26},
  {'transaction-id': 18117, 'amount': 22},
  {'transaction-id': 20855, 'amount': 23},
  {'transaction-id': 21198, 'amount': 24},
  {'transaction-id': 25098, 'amount': 24},
  {'transaction-id': 27115, 'amount': 25},
  {'transaction-id': 29519, 'amount': 22},
  {'transaction-i

In [131]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1284, 'amount': 28},
 {'transaction-id': 2262, 'amount': 24},
 {'transaction-id': 4392, 'amount': 25})

In [132]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(28, 24, 25)

In [133]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1392.7141634314305

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [134]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Frank', 'Edith', 'Alice'])]

In [135]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [8, 0, 6, 2, 4]), (1, [7, 9, 5, 1, 3])]

In [136]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [137]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [138]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 180), ('Alice', 180), ('Alice', 180), ('Alice', 180), ('Alice', 180), ('Alice', 180), ('Alice', 210), ('Alice', 210), ('Bob', 98), ('Bob', 102), ('Bob', 118), ('Bob', 118), ('Bob', 200), ('Bob', 200), ('Charlie', 132), ('Charlie', 132), ('Charlie', 132), ('Charlie', 132), ('Charlie', 132), ('Charlie', 154), ('Charlie', 286), ('Dan', 84), ('Dan', 84), ('Dan', 84), ('Dan', 84), ('Dan', 84), ('Dan', 98), ('Dan', 182), ('Edith', 100), ('Edith', 102), ('Edith', 102), ('Edith', 102), ('Edith', 102), ('Edith', 103), ('Edith', 118), ('Edith', 118), ('Frank', 114), ('Frank', 115), ('Frank', 116), ('Frank', 117), ('Frank', 129), ('Frank', 134), ('Frank', 228), ('George', 131), ('George', 132), ('George', 152), ('George', 154), ('George', 262), ('George', 263), ('Hannah', 138), ('Hannah', 138), ('Hannah', 138), ('Hannah', 138), ('Hannah', 138), ('Hannah', 138), ('Hannah', 161), ('Hannah', 161), ('Ingrid', 84), ('Ingrid', 84), ('Ingrid', 84), ('Ingrid', 84), ('Ingrid', 84), ('Ingrid', 8

In [139]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1500), ('Bob', 836), ('Charlie', 1100), ('Dan', 700), ('Edith', 847), ('Frank', 953), ('George', 1094), ('Hannah', 1150), ('Ingrid', 700), ('Jerry', 898), ('Kevin', 1191), ('Laura', 1200), ('Michael', 745), ('Norbert', 950), ('Oliver', 500), ('Patricia', 841), ('Quinn', 900), ('Ray', 750), ('Sarah', 1100), ('Tim', 950), ('Ursula', 850), ('Victor', 848), ('Wendy', 1250), ('Xavier', 1108), ('Yvonne', 850), ('Zelda', 998)]
Wall time: 1.82 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [ ]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [ ]:
df1 = js.to_dataframe()
df1.head()

This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [ ]:
%time df1.groupby('name').id.count().compute().head()

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [ ]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

In [ ]:
df = transactions.to_dataframe()
df.head()

In [ ]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

In [ ]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')
b.take(1)